# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
! pip install -q requests bs4 schedule pytest # установка библиотек, если ещё не установлены

In [1]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import re
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import schedule

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [2]:
 def get_book_data(book_url: str) -> dict:
    """
    Return a dictionary containing information about a book from
    toscrape.com site.

    Parameters
    ----------
    book_url : str
        URL of the book page. It must be full URL, e. g.
        "https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    """

    book_info = {}

    try:
        response = requests.get(book_url, timeout=5)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return {"error": str(e), "book_url": book_url}

    if response.status_code == 200:
        soup = bs(response.text, "html.parser")
        book_info["title"] = soup.find("h1").text

        for tr in soup.find("table").find_all("tr"):
            # .replace('\xc2', '') is to delete unexpected 'Â' symbol
            # preceding '£' symbol in a value of a book price
            book_info[tr.find("th").text] = tr.find("td").text.replace(
                '\xc2', '')

        product_description = soup.find(
            "div", attrs={"id": "product_description"}
            )
        
        if product_description is not None:
            book_info["product_description"] = \
                product_description.find_next_sibling("p").text

        book_info["star_rating"] = soup.find(
            "p", class_="star-rating")['class'][1]
    else:
        return {"error": f'Request failed with code {response.status_code}',
                "book_url": book_url}

    return book_info

In [3]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'UPC': 'a897fe39b1053632',
 'Product Type': 'Books',
 'Price (excl. tax)': '£51.77',
 'Price (incl. tax)': '£51.77',
 'Tax': '£0.00',
 'Availability': 'In stock (22 available)',
 'Number of reviews': '0',
 'product_description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and s

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [4]:
def scrape_books(is_save: bool = False, books_count: int = -1) -> dict:
    """
    Return information about several books from toscrape.com site.

    Result dictionary contains two elements: the 'books' element with
    information about books and the 'fault_pages' with information
    about some errors of scraping.

    Parameters
    ----------
    is_save : bool
        If True the result list is saved to books_data.txt file in the
        same folder where the running script or Jupyter Notebook is.
        Default value is False.
    books_count : int
        Desired count of books in result list. If -1 (by default) all
        the books will be proccessed.
    """

    result = {"books": [], "fault_pages": []}
    page_number = 1
    pages_in_total = 0
    base_url = 'https://books.toscrape.com/catalogue/'

    while True:
        page_url = f'http://books.toscrape.com/catalogue/page-{page_number}.html'

        try:
            response = requests.get(page_url, timeout=5)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            result["fault_pages"].append({"page_number": page_number,
                                          "error": str(e),
                                          "book_url": page_url})

        if response.status_code == 200:
            soup = bs(response.text, "html.parser")
            pages_in_total = int(re.search(r"\d+$", soup.find(
                "li", class_="current").text.strip()).group(0))

            for li in soup.find("ol", class_="row").find_all("li"):
                # Is it make sense to do a pause to not make the overload on the site?
                time.sleep(0.5)

                result["books"].append(get_book_data(
                    base_url + li.find("a")["href"]))

                if len(result["books"]) == books_count:
                    break
        else:
            result["fault_pages"].append(
                {"page_number": page_number,
                "error": f"Response status code was {response.status_code}",
                "book_url": page_url})

        if page_number == pages_in_total \
                or len(result["books"]) == books_count \
                or (len(result["books"]) == 0
                    and len(result["fault_pages"]) > 5):
            if is_save:
                try:
                    file_path = os.path.dirname(__file__) \
                        + '/books_data.txt'
                except NameError:
                    file_path = 'books_data.txt'

                with open(file_path, 'w', encoding='UTF-8') as f:
                    f.write(json.dumps(result))

            break

        page_number += 1

    return result

In [5]:
# Проверка работоспособности функции
res = scrape_books(is_save=True, books_count=3) # Допишите ваши аргументы
print(type(res), len(res["books"])) # и проверки

with open('../artifacts/books_data.txt', 'r') as f:
    data = json.load(f)

print(len(data["books"]))

<class 'dict'> 3
1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.every().day.at("16:39").do(scrape_books, is_save=True, books_count=5)

while True:
    schedule.run_pending()
    time.sleep(1)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [6]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! cd .. && pytest

============================= test session starts ==============================
platform linux -- Python 3.11.2, pytest-8.4.2, pluggy-1.6.0
rootdir: /home/v/Documents/v/Self development/Науки о данных/Python/hw3
plugins: anyio-4.11.0, cov-7.0.0
collected 2001 items                                                           

tests/test_scraper.py .................................................. [  2%]
........................................................................ [  6%]
........................................................................ [  9%]
........................................................................ [ 13%]
........................................................................ [ 16%]
........................................................................ [ 20%]
........................................................................ [ 24%]
........................................................................ [ 27%]
.................................

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```